# Create Functions

In [22]:
tools_function = [
    {
        "type": "function",
        "function": {
            "name": "get_countries",
            "description": "Retrieves the countries available."
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_travel_budget",
            "description": "Retrieves the travel budget for a country.",
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Retrieves the weather for a country.",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "The country (e.g., 'USA', 'France')."
                    }
                },
                "required": ["country"]
            }
        }
    },

]

# define functions
def get_countries():
    return """
        United States
        Australia
        France
    """

def get_travel_budget():
    return """

        United States Budget:
            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.

        Australia Budget:
            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
            Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.

        France Budget:
            Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.
        """
def get_weather(country):
    if country == "United States":
        return """
            The United States has a diverse climate. In general:
            - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
            - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).
        """
    elif country == "Australia":
        return """
            Australia experiences varied climates:
            - **Winter**: Mild in the north, cooler in the south. Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
            - **Summer**: Hot and dry in the interior, humid in the north. Average temperatures range from 68°F (20°C) in Hobart to 91°F (33°C) in Darwin.
        """
    elif country == "France":
        return """
            France has several climate zones:
            - **Winter**: Cold in the north, mild in the south. Average temperatures range from 35.5°F (2°C) in the northeast to 48°F (9°C) in the south.
            - **Summer**: Warm and sunny. Average temperatures range from 63°F (17°C) in the north to 77°F (25°C) along the Mediterranean coast.
        """
    else:
        return "Country not found"

In [23]:
from openai import AzureOpenAI
import os

from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

client = AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=azure_endpoint,
    api_key=api_key
)

# Create assistant
assistant = client.beta.assistants.create(
  name="Travel Assistant",
  instructions="""You are an assistant that answers questions about travel. 
  """,
  model=azure_deployment, 
  tools=tools_function
)

# Create thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_czcW4icdPHUT9iPXObtePyiI', created_at=1731424791, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [25]:
import time
import json


user_question ="""If i only have 300 USD, what country can I go to for 4 days?
"""

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=user_question
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  
  elif run.status == 'requires_action':
    # the assistant requires calling some functions and submit the tool outputs back to the run
    available_functions = {
      "get_countries": get_countries,
      "get_travel_budget": get_travel_budget,
      "get_weather": get_weather
    }
    tool_outputs = []
    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
          
      if function_name == "get_countries":
        function_response = get_countries()
      elif function_name == "get_travel_budget":
        function_response = get_travel_budget()
      elif function_name == "get_hotels_for_city":
        function_response = get_weather(country=function_args.get("country"))    
      else:
        function_response = "Unknown function"

      tool_outputs.append(
        { 
          "tool_call_id": tool_call.id,
          "output": str(function_response)
        }
      )
      run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs = tool_outputs
      )
  
    else:
      print(run.status)


# Print the assistant response
content_block = messages.data[0].content[0]
value = content_block.text.value
print(value)

Let's calculate the budget traveler costs in each available country for 4 days:

- **United States**: $121 per day * 4 days = $484
- **Australia**: $75 per day * 4 days = $300
- **France**: $100 per day * 4 days = $400

With a budget of $300, you can go to Australia for 4 days.


# Delete Assistant

In [26]:
response = client.beta.assistants.delete(assistant.id)